In [ ]:
prob.variables()

In [ ]:
dummyVariableNamesList5

In [57]:
#####################################################################
########      Creating initial parameters         ###################
#####################################################################
import matplotlib.pyplot as plt
%matplotlib inline 
from __future__ import division 
from pandasql import sqldf
import pandas as pd
import copy
import numpy as np
import inflect
from pulp import * 
from operator import add


# extract from a dataframe using a filter on week the desirable field 
def extract(data,week,field):
    return float(data[data.week == week][field])    
# a more general function to extract field2 data based on filtering on field1
def extract2(data,field1,condition,field2):
    return float(data[data[field1] == condition][field2])  

def plotter(objectList):
    fig, ax = plt.subplots(1, sharex=False, sharey=False)
    data = [(x, y.varValue) for x in objectList for y in prob.variables() if y.name.split('_')[1] == x ]
    fig.set_size_inches(30, 5)
    my_xticks = [i[0] for i in data]
    plt.xticks(range(len(data)), my_xticks)
    labels = ax.get_xticklabels()
    ax.set_xticklabels(labels, rotation=-30)
    plt.plot([i[1] for i in data])
    plt.show()
    
    
# run this at the end of the first optimizaiton when still current week is 1 
def headcountDfUpdater(currentWeek,headcountDf):
    # solving in the current week determines the hires 4 weeks from now
    # hiring decision for the first 3 weeks is frozen 
    week = currentWeek+3
    # converts numerical week to string week, for example 1 becomes 'one'
    weekWord = str(inflect.engine().number_to_words(week).replace(" ", "").replace("-", ""))
    hireType = ['AMZ','TEMP']
    hireDate = []
    headcountValue = []
    laborType = []
    for hireTypeIntsnace in hireType:
        employeeTarget = 'employee' + hireTypeIntsnace + weekWord
        for v in prob.variables():
            # if the decision variable name is 'employeeAMZfour' then add the value to the table 
            emplpyeeDecisionVariableName = str(v.name.split('_')[1])
            if emplpyeeDecisionVariableName == employeeTarget:
                hireDate.append(week)
                laborType.append(hireTypeIntsnace)
                headcountValue.append(float(v.varValue))
    newHeadcountDf = pd.DataFrame(np.column_stack((hireDate,headcountValue,laborType)),
                                    columns=['hireDate', 'headcountValue','laborType' ])
    frames = [headcountDf, newHeadcountDf]
    headcountDf = pd.concat(frames)
    hireDate2Integer = lambda x: int(x['hireDate'])
    headcountDf['hireDate'] = headcountDf.apply(hireDate2Integer,axis = 1)
    return headcountDf

# run this at the end of the first optimizaiton when still current week is 1 
def BacklogUpdater(currentWeek,backlogDf):
    for v in prob.variables():
        DecisionVariableNames = str(v.name).split('_')
        if (DecisionVariableNames[0] == 'backlog') & (DecisionVariableNames[1] == 'backlogWeek' + str(currentWeek)):
            week = [currentWeek]
            backlogValue = [v.varValue]
    newBacklogDf = pd.DataFrame(np.column_stack((week,backlogValue)),
                                columns=['week', 'backlogValue' ])
    frames = [backlogDf, newBacklogDf]
    BacklogDf = pd.concat(frames)
    backlogDf['week'] = backlogDf.apply(week2Integer,axis = 1)
    return BacklogDf

def labor_cost(currentWeek):
    weekString = str(inflect.engine().number_to_words(currentWeek).replace(" ", "").replace("-", "")) 
    laborCost = 0
    for v in prob.variables():
        DecisionVariableNames = str(v.name).split('_')
        if (DecisionVariableNames[0] == 'MOT') & (DecisionVariableNames[1] == 'MOT' + weekString):
            laborCost += (+1)*v.varValue*MOTClassNamesDict['MOT' + weekString].cost
        if (DecisionVariableNames[0] == 'VTO') & (DecisionVariableNames[1] == 'VTO' + weekString):
            laborCost += (-1)*v.varValue*VTOClassNamesDict['VTO' + weekString].cost
        if ('dummy' in DecisionVariableNames[0]) & (v.varValue > 0):
            return 10000000
#             break
    exec("laborCost += currentEmployeeCostWeek" + str(currentWeek))
    return laborCost

simulationHorizon = 52 # weeks to generate values 
simulations = ['attrition']
runs = 1
counter = 0 
costs = {}
costs2 = list()
backlogGraph = list()
hiringGraph = list()
solution = list()
for simulation in simulations:
#     costs = {}
    percentAttendance = np.linspace(0,0,runs)
    percentAttrition = np.linspace(0,0,runs)
    percentDemand = np.linspace(1,1,runs) 
    percentRate = np.linspace(1,1,runs) 
    if simulation == 'attendance':
        percentAttendance = np.linspace(0.02,-0.02,runs)*0
    if simulation == 'attrition':
        percentAttrition = np.linspace(-0.02,0.02,runs)*0
    if simulation == 'demand':
        percentDemand = np.linspace(.98,1.02,runs)
    if simulation == 'rate':
        percentRate = np.linspace(1.02,0.98,runs)
    for run in range(runs):
        print('atten',percentAttendance[run],'attr',percentAttrition[run],'dem',percentDemand[run],'rat',percentRate[run])
        counter += 1
        prob = ''
        print('run ',run,' for ', simulation)
        
        week =  list(range(-4,simulationHorizon)) + list(range(-4,simulationHorizon))
        headcount = np.linspace(100,100,len(week))
#         attritionCount = [8]*int(len(week)/2) + [1]*int(len(week)/2)
        attritionCount = list(map(add,[10]*int(len(week)/2) + [2]*int(len(week)/2),
                         list(np.random.normal(0,1,len(week)/2)*2) + list(np.random.normal(0,1,len(week)/2))))
        laborType = ['TEMP']*int(len(week)/2) + ['AMZ']*int(len(week)/2)
        attritionActuals = pd.DataFrame(np.column_stack((week,headcount,attritionCount,laborType)),
                                        columns=['week','headcount', 'attritionCount',  'laborType'])
        attritionRateCalculator = lambda x: float(x['attritionCount'])/float(x['headcount'])
        attritionActuals['actualRate'] = attritionActuals.apply(attritionRateCalculator,axis = 1)
        week2Integer = lambda x: int(x['week'])
        attritionActuals['week'] = attritionActuals.apply(week2Integer,axis = 1)
        pysqldf = lambda q: sqldf(q, globals())
        # attrition inputs 
        attritionForecastsDf =  pysqldf("""
        SELECT 
         a.week
        ,a.laborType
        ,a.attritionCount
        ,a.headcount
        ,a.actualRate
        ,SUM(b.attritionCount)/SUM(b.headcount) AS predictedRate
        ,a.actualRate - SUM(b.attritionCount)/SUM(b.headcount) AS error
        FROM attritionActuals a 
        LEFT JOIN attritionActuals b
        ON (a.laborType = b.laborType)
        AND b.week between (a.week - 4) and (a.week - 1) 
        GROUP BY a.week,a.laborType
        order by a.week ; """)
    #     print(attritionForecastsDf)
    # made sure the rates do not go below 0. 
        attritionForecastsDf.predictedRate = attritionForecastsDf.predictedRate + percentAttrition[run] + attritionForecastsDf.error*0.0
        print('attrition error :',abs(attritionForecastsDf.error).mean()*100)

        hoursScheduled = [100]*int(len(week)/2) + [100]*int(len(week)/2)
        hoursWorked = list(map(add,[70.0]*int(len(week)/2) + [80.0]*int(len(week)/2),
                         list(np.random.normal(0,3,len(week)/2)*2) + list(np.random.normal(0,2,len(week)/2))))
        laborType = ['TEMP']*int(len(week)/2) + ['AMZ']*int(len(week)/2)
        attendanceActuals = pd.DataFrame(np.column_stack((week,hoursWorked,hoursScheduled,laborType)),
                                        columns=['week', 'hoursWorked', 'hoursScheduled','laborType' ])
        attendanceRateCalculator = lambda x: float(x['hoursWorked'])/float(x['hoursScheduled'])
        attendanceActuals['actualRate'] = attendanceActuals.apply(attendanceRateCalculator,axis = 1)
        attendanceActuals['week'] = attendanceActuals.apply(week2Integer,axis = 1)
        pysqldf = lambda q: sqldf(q, globals())
        # attendance inputs 
        attendanceForecastsDf =  pysqldf("""
        SELECT 
         a.week
        ,a.laborType
        ,a.hoursWorked
        ,a.hoursScheduled
        ,a.actualRate
        ,SUM(b.hoursWorked)/SUM(b.hoursScheduled) AS predictedRate
        ,a.actualRate - SUM(b.hoursWorked)/SUM(b.hoursScheduled) AS error
        FROM attendanceActuals a 
        LEFT JOIN attendanceActuals b
        ON (a.laborType = b.laborType)
        AND b.week between (a.week - 4) and (a.week - 1) 
        GROUP BY a.week,a.laborType
        order by a.week ; """)
        attendanceForecastsDf.predictedRate = attendanceForecastsDf.predictedRate + percentAttendance[run]
        print('attendance error :',abs(attendanceForecastsDf.error).mean()*100)
        # learning curve rates 
        learningCurveValues = list(np.linspace(0.65,1.00,int(len(week)/2))) + list(np.linspace(0.65,1.00,int(len(week)/2)))
        laborType = ['TEMP']*int(len(week)/2) + ['AMZ']*int(len(week)/2)
        learningCurveDf = pd.DataFrame(np.column_stack((week,learningCurveValues,laborType)),
                                        columns=['week', 'learningCurveValue','laborType' ])
        learningCurveDf['week'] = learningCurveDf.apply(week2Integer,axis = 1)
        # veteran rates and labor cost 
        laborCost = [15,20]
        laborType = ['TEMP','AMZ']
        veteranRateValues = [131*percentRate[run] ,131*percentRate[run]]
        laborTypeDf = pd.DataFrame(np.column_stack((laborType,veteranRateValues,laborCost)),
                                        columns=['laborType' ,'veteranRate','laborCost'])
        # current employee headcount and their hiring dates 
        hireDate =  list(range(-10,4)) + list(range(-10,4))
        headcountValue = np.linspace(15,15,len(hireDate))
        headcountValue[0] = 270  # amazonian veterans
        headcountValue[1] = 250  # amazonian veterans 
        laborType =  ['AMZ']*int(len(hireDate)/2) + ['TEMP']*int(len(hireDate)/2) 
        headcountDf = pd.DataFrame(np.column_stack((hireDate,headcountValue,laborType)),
                                        columns=['hireDate', 'headcountValue','laborType' ])
        hireDate2Integer = lambda x: int(x['hireDate'])
        headcountDf['hireDate'] = headcountDf.apply(hireDate2Integer,axis = 1)
#         print(headcountDf)
        # demand 
        week =  range(-4,simulationHorizon)
    #     demandActual = np.linspace(2000000,2000000,len(week))
        np.random.seed(1234)
        demandActual = np.linspace(3000000*percentDemand[run],3000000*percentDemand[run],len(week))
        demandForecast = demandActual + np.random.normal(0,demandActual[0]*0.05,len(week))*1 
        
        demandDf = pd.DataFrame(np.column_stack((week,demandActual,demandForecast)),
                                        columns=['week', 'demandActual','demandForecast' ])
        demandDf['week'] = demandDf.apply(week2Integer,axis = 1)
        # initial backlog  
        backlogValue = [0]
        week = [0]
        backlogDf = pd.DataFrame(np.column_stack((week,backlogValue)),
                                        columns=['week', 'backlogValue' ])
        backlogDf['week'] = backlogDf.apply(week2Integer,axis = 1)



    #####################################################################
    ###############  Creating Decision Variable Calsses #################
    #####################################################################
    # hireDate = the hiring date (week number) of the employee group of interest 
    # targetWeek = the date at which the survival value is requested
    # currentWeek = the week in which the calculation is made (this determines what prediction to use)
    # laborType = temp or amz 
        def survivalDataFrame(currentWeek,attritionForecastsDf,improvement):            
            attritionUpdatedDf = pysqldf("""
                SELECT 
                 a.week
                ,a.laborType
                ,a.attritionCount
                ,a.headcount
                ,a.actualRate
                ,CASE 
                    WHEN a.week <= """ + str(currentWeek) + """ THEN a.actualRate
                    ELSE b.predictedRate + """ + str(improvement) + """*(a.actualRate - b.predictedRate)  END as predictedRate
                ,CASE 
                    WHEN a.week <= """ + str(currentWeek) + """ THEN a.actualRate - a.predictedRate
                    ELSE a.actualRate - b.predictedRate END as error
                FROM attritionForecastsDf a
                LEFT JOIN (SELECT * FROM attritionForecastsDf b WHERE b.week = """ + str(currentWeek) + """) b
                ON a.laborType = b.laborType 
                order by a.week ; """) 
            return attritionUpdatedDf

        def attendanceDataFrame(currentWeek,attendanceForecastsDf,improvement):            
            attendanceUpdatedDf = pysqldf("""
                SELECT 
                 a.week
                ,a.laborType
                ,a.hoursWorked
                ,a.hoursScheduled
                ,a.actualRate
                ,CASE 
                    WHEN a.week <= """ + str(currentWeek) + """ THEN a.actualRate
                    ELSE b.predictedRate + """ + str(improvement) + """*(a.actualRate - b.predictedRate)  END as predictedRate
                ,CASE 
                    WHEN a.week <= """ + str(currentWeek) + """ THEN a.actualRate - a.predictedRate
                    ELSE a.actualRate - b.predictedRate END as error
                FROM attendanceForecastsDf a
                LEFT JOIN (SELECT * FROM attendanceForecastsDf b WHERE b.week = """ + str(currentWeek) + """) b
                ON a.laborType = b.laborType 
                order by a.week ; """) 

            return attendanceUpdatedDf
        # laborType = temp or amz 
        def survivalFunc(hireDate,targetWeek,currentWeek,laborType,attritionForecastsDf): # prediction depends on currentWeek
            data = survivalDataFrame(currentWeek,attritionForecastsDf,0)
            data = data[data.laborType == laborType]
            if hireDate <= 0:  # we do not apply attrition to weeks prior to week 1
                if targetWeek <= 0:
                    return 1
                if targetWeek > 0:
                    return np.prod(1- data[(data.week <= targetWeek)&(data.week > 0)].predictedRate)
            if hireDate <= targetWeek :
                # prediction value depend on currentWeek , compounding depends on targetWeek
                return np.prod(1- data[(data.week <= targetWeek)&(data.week >= hireDate)].predictedRate)
        #                 return (1- data[data.week == currentWeek].predictedRate)**(targetWeek - hireDate + 1)
            if hireDate > targetWeek :
                return 0

        def attendanceFunc(hireDate,targetWeek,currentWeek,laborType,attendanceForecastsDf): # prediction depends on currentWeek
            data = attendanceDataFrame(currentWeek,attendanceForecastsDf,0)
            data = data[data.laborType == laborType]
            if hireDate <= 0:  # attendance prior to week 1 is not considered - do not call for values prior to week 1
                if targetWeek <= 0:
                    return 1
                if targetWeek > 0:
                    return data[data.week == targetWeek].predictedRate
            if hireDate <= targetWeek :
                return data[data.week == targetWeek].predictedRate
            if hireDate > targetWeek :
                return 0


        def learningCurveFunc(hireDate,targetWeek,laborType):  # rate does not depend on current week 
                data = learningCurveDf[learningCurveDf.laborType == laborType]
                if targetWeek - hireDate > 10:
                    return 1
                if targetWeek < hireDate: # if you have not been hired yet, no learning curve 
                    return 0
                else:
                    return float(data[data.week == int(targetWeek - hireDate)].learningCurveValue)

        def veteranRateFunc(laborType):  # rate does not depend on current week 
                return float(laborTypeDf[laborTypeDf.laborType == laborType].veteranRate)    

        # class employee keeps track of a cohort information through time using methods 
        class employee():
            Schedulehours = 40
            def __init__(self,laborType = '',hireDate = 0,regularCost = 0,initialHeadcount = 0,mainProcess = [],currentWeek = 1, name = 'empty'):
                self.laborType = laborType
                self.hireDate = hireDate
                self.regularCost = regularCost
                self.initialHeadcount = initialHeadcount
                self.mainProcess = mainProcess
                self.currentWeek = currentWeek
                self.name = name
            def effectiveSurvival(self,targetWeek):
                return float(survivalFunc(self.hireDate,targetWeek,self.currentWeek,self.laborType,attritionForecastsDf))

            def effectiveAttendance(self,targetWeek):
                return float(attendanceFunc(self.hireDate,targetWeek,self.currentWeek,self.laborType,attendanceForecastsDf))

            def effectiveHeadcountCoef(self,targetWeek):
                survived = self.effectiveSurvival(targetWeek)
                attended = self.effectiveAttendance(targetWeek)
                return float(survived*attended)        

            def effectiveHeadcount(self,targetWeek):
                E = self.effectiveHeadcountCoef(targetWeek)
                return float(self.initialHeadcount*E)

            def showHoursCoef(self,targetWeek):
                return float(self.effectiveHeadcountCoef(targetWeek)*self.Schedulehours)

            def showHours(self,targetWeek):
                return float(self.showHoursCoef(targetWeek)*self.initialHeadcount)

            def learningCurve(self,targetWeek):
                return learningCurveFunc(self.hireDate,targetWeek,self.laborType)

            def veteranRate(self):
                return float(veteranRateFunc(self.laborType))
            
            def rate(self,targetWeek):
                return float(self.learningCurve(targetWeek)*self.veteranRate())

            def capacityCoef(self,targetWeek):
                showHourCoefValue = self.showHoursCoef(targetWeek)
                learningCurveValue = self.learningCurve(targetWeek)
                veteranRateValue = self.veteranRate()
                return float(showHourCoefValue*learningCurveValue*veteranRateValue)

            def capacity(self,targetWeek):
                return float(self.capacityCoef(targetWeek)*self.initialHeadcount)

            def costCoef(self,targetWeek):
                if self.hireDate <= 0: 
                    if targetWeek <= 0:
                        return 0
                    if targetWeek > 0:
                        return float(self.regularCost*self.showHoursCoef(targetWeek))
                if self.hireDate > targetWeek:
                    return 0
                if self.hireDate <= targetWeek:
                    return float(self.regularCost*self.showHoursCoef(targetWeek))

            def cost(self,targetWeek):
                return float(self.costCoef(targetWeek)*self.initialHeadcount)


            def vetCoef(self,targetWeek):
                if targetWeek - 10 >= self.hireDate :
                    return 1
                else:
                    return 0

            def vetHoursCoef(self,targetWeek):
                return self.vetCoef(targetWeek)*self.showHoursCoef(targetWeek)

            def vetHours(self,targetWeek):
                return self.vetCoef(targetWeek)*self.showHours(targetWeek)

            def vetCapacity(self,targetWeek):
                return self.vetHours(targetWeek)*self.veteranRate()
    # embassadors spent 1 full day with new hires, then follow up with 5 of them for up to 2 days 
            def trainingCapacityCoef(self,targetWeek):
                if self.hireDate == targetWeek and self.hireDate > 0:
                    vetTraining = 3*self.veteranRate()  # we lose 3 vet hour of vets capacity per new hire  
                    newHireTraining = self.learningCurve(targetWeek)*self.veteranRate()*2 #2 hours of new hire went on safety training 
                    return float(vetTraining+newHireTraining)
                else:
                    return 0  #current head-count do not need training , training only occures in week 1
 #connect this to the prevoious method 
            def trainingVetHourCoef(self,targetWeek):
                if self.hireDate == targetWeek and self.hireDate > 0:
                    vetTraining = 3 # we lose 1 hour of vets per new hire 
                    return float(vetTraining)
                else:
                    return 0  #current head-count do not need training , training only occures in week 1

        class MOT():
            cost = (extract2(laborTypeDf,'laborType','AMZ','laborCost')+
                    extract2(laborTypeDf,'laborType','TEMP','laborCost'))*(1.5/2)  # average times 1.5 
            rate = extract2(laborTypeDf,'laborType','AMZ','veteranRate')
            def __init__(self,name ='', targetWeek = 0):
                self.name = name 
                self.targetWeek= targetWeek  # the week you call MOT
            def hourIndicator(self,week):
                if self.targetWeek == week:
                    return 1
                else: 
                    return 0 

        class VTO():
            cost = extract2(laborTypeDf,'laborType','TEMP','laborCost')
            rate = extract2(laborTypeDf,'laborType','AMZ','veteranRate')    
            def __init__(self,name ='', targetWeek = 0):
                self.name = name 
                self.targetWeek = targetWeek   # the week you call VTO
            def hourIndicator(self,week):
                if self.targetWeek == week:
                    return 1
                else: 
                    return 0 


        class laborShare(): 
            def __init__(self,name ='', targetWeek = 0):
                self.name = name 
                self.targetWeek = targetWeek 
            def hourIndicator(self,week):
                if self.targetWeek == week:
                    return 1
                else: 
                    return 0 



# #####################################################################
# ###############  ROUND 2 OPTIMIZATION WEEK 2 #################
# #####################################################################
# ### ********** to do ******* #######
totalCost = 0
# #####################################################################
# ###############        creating class instances      ################
# #####################################################################
for simulationWeeks in range(1,2):
    print('running optimization for week {} .... '.format(simulationWeeks))
    prob = ''
    # the current week or the week when we are solving for labor
    currentWeek = simulationWeeks

    # for current employees we have their headcount
    employeeStatus = ['current','future']
    for employeeStatusItem in employeeStatus:
        code = compile( employeeStatusItem + 'Employee' + 'ClassList = list()',  '<string>', 'exec')  
        exec(code)

    # iterate throught the table of current headcount and generate classes 
    # stores all the employee objects into a list 
    for index,row in headcountDf[(headcountDf.hireDate <= currentWeek + 2)].iterrows(): 
        name = """employee""" + str(row.laborType.upper()) + str(inflect.engine().number_to_words(row.hireDate).replace(" ", "").replace("-", "")) 
        code = compile( name  + """ = employee(
                                            laborType = """ + """str(row.laborType)""" + """,
                                            hireDate = """ + str(row.hireDate) + """,
                                            regularCost = """ + str(extract2(laborTypeDf,'laborType',str(row.laborType),'laborCost'))  + """,  
                                            initialHeadcount = """ + str(row.headcountValue) + """,
                                            mainProcess = [],
                                            currentWeek = """ + str(currentWeek) + """,
                                            name = '""" + name  + """')""",
                                            '<string>', 'exec')
        exec(code)
        code = compile( 'currentEmployeeClassList.append' + '(' +name+ ')',
                                            '<string>', 'exec')   
        exec(code)
    # generat hiring variables for new hires - hiring is freezed in the first 3 weeks  
    for hireDate in range(currentWeek + 3,currentWeek + 13):
        for laborType in ['AMZ','TEMP']:
                name = """employee""" + str(laborType.upper()) + str(inflect.engine().number_to_words(hireDate).replace(" ", "").replace("-", "")) 
                code = compile( name  + """ = employee(
                                                    laborType = laborType,
                                                    hireDate = """ + str(hireDate) + """,
                                                    regularCost = """ + str(extract2(laborTypeDf,'laborType',laborType,'laborCost'))  + """,    
                                                    initialHeadcount = 0,
                                                    mainProcess = [],
                                                    currentWeek = """ + str(currentWeek) + """,
                                                    name = '""" + name  + """')""",
                                                    '<string>', 'exec')
                exec(code)
                code = compile( 'futureEmployeeClassList.append' + '(' +name+ ')',
                                                    '<string>', 'exec')   
                exec(code)

    # generat VTO classes for each week 
    VTOClassList = list()  # list of objects 
    for week in range(currentWeek ,currentWeek + 13):
        for timeLever in ['VTO']:
            name = timeLever + str(inflect.engine().number_to_words(week).replace(" ", "").replace("-", "")) 
            code = compile( (name  + """ = """ + timeLever +
                                                """(name = '""" + name + 
                                                """', targetWeek = week)"""),
                                                '<string>', 'exec')
            exec(code)
            code = compile( 'VTOClassList.append' + '(' +name+ ')',
                                                '<string>', 'exec')   
            exec(code)


    # generat MOT classes for each week 
    MOTClassList = list()  # list of objects 
    for week in range(currentWeek ,currentWeek + 13):
        for timeLever in ['MOT']:
            name = timeLever + str(inflect.engine().number_to_words(week).replace(" ", "").replace("-", "")) 
            code = compile( (name  + """ = """ + timeLever +
                                                """(name = '""" + name + 
                                                """', targetWeek = week)"""),
                                                '<string>', 'exec')
            exec(code)
            code = compile( 'MOTClassList.append' + '(' +name+ ')',
                                                '<string>', 'exec')   
            exec(code)

    # generat laborShare classes for each week 
    laborShareClassList = list()  # list of objects 
    for week in range(currentWeek ,currentWeek + 13):
        name = 'laborShare' + str(inflect.engine().number_to_words(week).replace(" ", "").replace("-", "")) 
        code = compile( (name  + """ = laborShare(
                                            name = '""" + name + """' 
                                           ,targetWeek = week)"""),
                                            '<string>', 'exec')
        exec(code)
        code = compile( 'laborShareClassList.append' + '(' +name+ ')',
                                            '<string>', 'exec')
        exec(code)

    #####################################################################
    ##     creating lists of names of the objects created earlier   #####
    #####################################################################

    # create a list of names of the instances of classes created earlier 
    # create a dictionary in which the key is the name of the class instance and the value is the class instance 
    futureEmployeeClassNamesList = list()
    futureEmployeeClassNamesDict = {}
    for variables in futureEmployeeClassList:
        futureEmployeeClassNamesList.append(variables.name)
        futureEmployeeClassNamesDict[variables.name] = variables 
    # create a list of names of the instances of classes created earlier 
    # create a dictionary in which the key is the name of the class instance and the value is the class instance 
    currentEmployeeClassNamesList = list()
    currentEmployeeClassNamesDict = {}
    for variables in currentEmployeeClassList:
        currentEmployeeClassNamesList.append(variables.name)
        currentEmployeeClassNamesDict[variables.name] = variables 
    # create a list of names of the instances of classes created earlier 
    # create a dictionary in which the key is the name of the class instance and the value is the class instance 
    MOTClassNamesList = list()
    MOTClassNamesDict = {}
    for variables in MOTClassList:
        MOTClassNamesList.append(variables.name) 
        MOTClassNamesDict[variables.name] = variables 
    # create a list of names of the instances of classes created earlier 
    # create a dictionary in which the key is the name of the class instance and the value is the class instance         
    VTOClassNamesList = list()
    VTOClassNamesDict = {}
    for variables in VTOClassList:
        VTOClassNamesList.append(variables.name) 
        VTOClassNamesDict[variables.name] = variables 
    # create a list of names of the instances of classes created earlier 
    # create a dictionary in which the key is the name of the class instance and the value is the class instance         
    laborShareClassNamesList = list()
    laborShareClassNamesDict = {}
    for variables in laborShareClassList:
        laborShareClassNamesList.append(variables.name)    
        laborShareClassNamesDict[variables.name] = variables 

    ################################################################################
    ##     creating dictionaries capacity, cost for current and futrue employees ###
    ################################################################################

    optimizationHorizon = range(currentWeek,currentWeek+13)

    # creating dictionaries with 
    # keys = futureEmployeeCapacityWeek + week + decision variable name  
    # values = coeficient of capacity (h * coeficient = capacity)
    for week in optimizationHorizon:
        exec(compile( 'futureEmployeeCapacityWeek' + str(week) + '={}','<string>', 'exec'))
        for variables in futureEmployeeClassList:
            exec(compile( 'futureEmployeeCapacityWeek' + str(week) + '[variables.name] = variables.capacityCoef(' + str(week) + ')'                     
                         ,'<string>', 'exec'))

    # weekly capacity of current emplpoyees     
    # keys = currentEmployeeCapacityWeek  +  week
    # value = sum of all processing capacity of current hires (plus the first three weeks hires)
    for week in optimizationHorizon:
        exec(compile( 'currentEmployeeCapacityWeek' + str(week) + '={}','<string>', 'exec'))
        s = sum([variables.capacity(week) for variables in currentEmployeeClassList])
        exec(compile( 'currentEmployeeCapacityWeek' + str(week) + ' = s '                   
                         ,'<string>', 'exec'))


    # weekly cost of future emplpoyees  
    # name = per week for example for week 1 a dictionary is generated with name futureEmployeeCostWeek1
    # keys = employee name  for example employeeAMZfour
    # value = sum of all processing capacity of current hires (plus the first three weeks hires) on that week 
    for week in optimizationHorizon:
        exec(compile( 'futureEmployeeCostWeek' + str(week) + '={}','<string>', 'exec'))
        for variables in futureEmployeeClassList:
            exec(compile( 'futureEmployeeCostWeek' + str(week) + '[variables.name] = variables.costCoef(' + str(week) + ')'                     
                         ,'<string>', 'exec'))

    # weekly cost of current emplpoyees  
    # name = per week for example for week 1 a variable is generated with name currentEmployeeCostWeek1
    # value = sum of all processing capacity of current hires (plus the first three weeks hires) on that week 
    for week in optimizationHorizon:
        s = sum([variables.cost(week) for variables in currentEmployeeClassList])
        exec(compile( 'currentEmployeeCostWeek' + str(week) + ' = s '                   
                         ,'<string>', 'exec'))

    # weekly total veteran hour coef of future employees 
    # there is one dictionary per week with name :  futureEmployeeVeteranHourWeek + week
    # each dictionary has :
    # key : employee class name
    # value : cost of that employee class in the specified week 
    for week in optimizationHorizon:
        exec(compile( 'futureEmployeeVeteranHourWeek' + str(week) + '={}','<string>', 'exec'))
        for variables in futureEmployeeClassList:
            exec(compile( 'futureEmployeeVeteranHourWeek' + str(week) + '[variables.name] = variables.vetHoursCoef(' + str(week) + ')'                     
                         ,'<string>', 'exec'))

    # weekly total veteran hour from current employees 
    # name = per week for example for week 1 a variable is generated with name urrentEmployeeVeteranHourWeek1
    # value = sum of all veteran hours from the current employees 
    for week in optimizationHorizon:
        s = sum([variables.vetHours(week) for variables in currentEmployeeClassList])
        exec(compile( 'currentEmployeeVeteranHourWeek' + str(week) + ' = s '                   
                         ,'<string>', 'exec'))

    for week in optimizationHorizon:
        s = sum([variables.effectiveHeadcount(week)*variables.rate(week) for variables in currentEmployeeClassList])
        s2 = sum([variables.effectiveHeadcount(week) for variables in currentEmployeeClassList])
        exec(compile( 'currentEmployeeRateWeek' + str(week) + ' = s/s2 '                   
                         ,'<string>', 'exec'))
    ###########################################################
    ##     creating decision variables for optimization    ####
    ###########################################################

    futureEmployeeDecisionVariables = LpVariable.dicts("futureEmployee",futureEmployeeClassNamesList,lowBound=0,upBound=100)
    MOTDecisionVariables = LpVariable.dicts("MOT",MOTClassNamesList,lowBound=0,upBound=10000000)
    VTODecisionVariables = LpVariable.dicts("VTO",VTOClassNamesList,lowBound=0,upBound=10000000)
    laborShareDecisionVariables = LpVariable.dicts("laborShare",laborShareClassNamesList,lowBound=0,upBound=10000000)
    endWeekBacklogVariableNamesList = ['backlogWeek' + str(i) for i in optimizationHorizon]
    endWeekBacklogDecisionVariables = LpVariable.dicts("backlog",endWeekBacklogVariableNamesList,lowBound=0,upBound=None)    
    dummyVariableNamesListCapacity = ['dummyCapacityWeek' + str(i) for i in optimizationHorizon]
    dummyDecisionVariablesCapacity = LpVariable.dicts("dummyCapacity",dummyVariableNamesListCapacity,lowBound=0,upBound=None)    
    dummyVariableNamesListProcess = ['dummyProcessWeek' + str(i) for i in optimizationHorizon]
    dummyDecisionVariablesProcess = LpVariable.dicts("dummyProcess",dummyVariableNamesListProcess,lowBound=0,upBound=None)    
    dummyVariableNamesListBacklog = ['dummyBacklogWeek' + str(i) for i in optimizationHorizon]
    dummyDecisionVariablesBacklog = LpVariable.dicts("dummyBacklog",dummyVariableNamesListBacklog,lowBound=0,upBound=None)    
    dummyVariableNamesListMOT = ['dummyMOTWeek' + str(i) for i in optimizationHorizon]
    dummyDecisionVariablesMOT = LpVariable.dicts("dummyMOT",dummyVariableNamesListMOT,lowBound=0,upBound=None)  
    dummyVariableNamesListVTO = ['dummyVTOWeek' + str(i) for i in optimizationHorizon]
    dummyDecisionVariablesVTO = LpVariable.dicts("dummyVTO",dummyVariableNamesListVTO,lowBound=0,upBound=None)  
    dummyVariableNamesListLaborshare = ['dummyLaborshareWeek' + str(i) for i in optimizationHorizon]
    dummyDecisionVariablesLaborshare = LpVariable.dicts("dummyLaborshare",dummyVariableNamesListLaborshare,lowBound=0,upBound=None)  
    ###################################################
    ##     Defining the optimization function      ####
    ###################################################
    prob = LpProblem("Labor Planning Basic",LpMinimize)
    ###################################################
    ##     Defining the objective                  ####
    ###################################################
#     print(headcountDf)
    print('generating objective function ....')
    exec("""prob += lpSum([futureEmployeeCostWeek"""+ str(currentWeek) +"""[i]*futureEmployeeDecisionVariables[i] for i in futureEmployeeClassNamesList ] +
                  [futureEmployeeCostWeek"""+ str(currentWeek+1) +"""[i]*futureEmployeeDecisionVariables[i] for i in futureEmployeeClassNamesList ] +
                  [futureEmployeeCostWeek"""+ str(currentWeek+2) +"""[i]*futureEmployeeDecisionVariables[i] for i in futureEmployeeClassNamesList ] +
                  [futureEmployeeCostWeek"""+ str(currentWeek+3) +"""[i]*futureEmployeeDecisionVariables[i] for i in futureEmployeeClassNamesList ] +
                  [futureEmployeeCostWeek"""+ str(currentWeek+4) +"""[i]*futureEmployeeDecisionVariables[i] for i in futureEmployeeClassNamesList ] +
                  [futureEmployeeCostWeek"""+ str(currentWeek+5) +"""[i]*futureEmployeeDecisionVariables[i] for i in futureEmployeeClassNamesList ] +
                  [futureEmployeeCostWeek"""+ str(currentWeek+6) +"""[i]*futureEmployeeDecisionVariables[i] for i in futureEmployeeClassNamesList ] +
                  [futureEmployeeCostWeek"""+ str(currentWeek+7) +"""[i]*futureEmployeeDecisionVariables[i] for i in futureEmployeeClassNamesList ] +
                  [futureEmployeeCostWeek"""+ str(currentWeek+8) +"""[i]*futureEmployeeDecisionVariables[i] for i in futureEmployeeClassNamesList ] +
                  [futureEmployeeCostWeek"""+ str(currentWeek+9) +"""[i]*futureEmployeeDecisionVariables[i] for i in futureEmployeeClassNamesList ] +
                  [futureEmployeeCostWeek"""+ str(currentWeek+10) +"""[i]*futureEmployeeDecisionVariables[i] for i in futureEmployeeClassNamesList ] +
                  [futureEmployeeCostWeek"""+ str(currentWeek+11) +"""[i]*futureEmployeeDecisionVariables[i] for i in futureEmployeeClassNamesList ] +
                  [futureEmployeeCostWeek"""+ str(currentWeek+12) +"""[i]*futureEmployeeDecisionVariables[i] for i in futureEmployeeClassNamesList ] +
                  [globals()[i].hourIndicator(week = """+ str(currentWeek) +""")*globals()[i].cost*(-1)*VTODecisionVariables[i] for i in VTOClassNamesList] +
                  [globals()[i].hourIndicator(week = """+ str(currentWeek+1) +""")*globals()[i].cost*(-1)*VTODecisionVariables[i] for i in VTOClassNamesList] +  
                  [globals()[i].hourIndicator(week = """+ str(currentWeek+2) +""")*globals()[i].cost*(-1)*VTODecisionVariables[i] for i in VTOClassNamesList] +
                  [globals()[i].hourIndicator(week = """+ str(currentWeek+3) +""")*globals()[i].cost*(-1)*VTODecisionVariables[i] for i in VTOClassNamesList] +  
                  [globals()[i].hourIndicator(week = """+ str(currentWeek+4) +""")*globals()[i].cost*(-1)*VTODecisionVariables[i] for i in VTOClassNamesList] +
                  [globals()[i].hourIndicator(week = """+ str(currentWeek+5) +""")*globals()[i].cost*(-1)*VTODecisionVariables[i] for i in VTOClassNamesList] +  
                  [globals()[i].hourIndicator(week = """+ str(currentWeek+6) +""")*globals()[i].cost*(-1)*VTODecisionVariables[i] for i in VTOClassNamesList] +
                  [globals()[i].hourIndicator(week = """+ str(currentWeek+7) +""")*globals()[i].cost*(-1)*VTODecisionVariables[i] for i in VTOClassNamesList] +  
                  [globals()[i].hourIndicator(week = """+ str(currentWeek+8) +""")*globals()[i].cost*(-1)*VTODecisionVariables[i] for i in VTOClassNamesList] +
                  [globals()[i].hourIndicator(week = """+ str(currentWeek+9) +""")*globals()[i].cost*(-1)*VTODecisionVariables[i] for i in VTOClassNamesList] +
                  [globals()[i].hourIndicator(week = """+ str(currentWeek+10) +""")*globals()[i].cost*(-1)*VTODecisionVariables[i] for i in VTOClassNamesList] +  
                  [globals()[i].hourIndicator(week = """+ str(currentWeek+11) +""")*globals()[i].cost*(-1)*VTODecisionVariables[i] for i in VTOClassNamesList] +
                  [globals()[i].hourIndicator(week = """+ str(currentWeek+12) +""")*globals()[i].cost*(-1)*VTODecisionVariables[i] for i in VTOClassNamesList] +  
                  [globals()[i].hourIndicator(week =  """+ str(currentWeek) +""")*globals()[i].cost*MOTDecisionVariables[i] for i in MOTClassNamesList] +
                  [globals()[i].hourIndicator(week =  """+ str(currentWeek+1) +""")*globals()[i].cost*MOTDecisionVariables[i] for i in MOTClassNamesList] + 
                  [globals()[i].hourIndicator(week =  """+ str(currentWeek+2) +""")*globals()[i].cost*MOTDecisionVariables[i] for i in MOTClassNamesList] +    
                  [globals()[i].hourIndicator(week =  """+ str(currentWeek+3) +""")*globals()[i].cost*MOTDecisionVariables[i] for i in MOTClassNamesList] +
                  [globals()[i].hourIndicator(week =  """+ str(currentWeek+4) +""")*globals()[i].cost*MOTDecisionVariables[i] for i in MOTClassNamesList] + 
                  [globals()[i].hourIndicator(week =  """+ str(currentWeek+5) +""")*globals()[i].cost*MOTDecisionVariables[i] for i in MOTClassNamesList] +
                  [globals()[i].hourIndicator(week =  """+ str(currentWeek+6) +""")*globals()[i].cost*MOTDecisionVariables[i] for i in MOTClassNamesList] + 
                  [globals()[i].hourIndicator(week =  """+ str(currentWeek+7) +""")*globals()[i].cost*MOTDecisionVariables[i] for i in MOTClassNamesList] +
                  [globals()[i].hourIndicator(week =  """+ str(currentWeek+8) +""")*globals()[i].cost*MOTDecisionVariables[i] for i in MOTClassNamesList] + 
                  [globals()[i].hourIndicator(week =  """+ str(currentWeek+9) +""")*globals()[i].cost*MOTDecisionVariables[i] for i in MOTClassNamesList] +
                  [globals()[i].hourIndicator(week =  """+ str(currentWeek+10) +""")*globals()[i].cost*MOTDecisionVariables[i] for i in MOTClassNamesList] + 
                  [globals()[i].hourIndicator(week =  """+ str(currentWeek+11) +""")*globals()[i].cost*MOTDecisionVariables[i] for i in MOTClassNamesList] +
                  [globals()[i].hourIndicator(week =  """+ str(currentWeek+12) +""")*globals()[i].cost*MOTDecisionVariables[i] for i in MOTClassNamesList] +  
                  [0.00001*endWeekBacklogDecisionVariables[i] for i in endWeekBacklogVariableNamesList ] + 
                  [0.00001*laborShareDecisionVariables[i] for i in laborShareClassNamesList ] + 
                  [100000000*dummyDecisionVariablesCapacity[i] for i in dummyVariableNamesListCapacity] + 
                  [100000000*dummyDecisionVariablesProcess[i] for i in dummyVariableNamesListProcess] + 
                  [100000000*dummyDecisionVariablesBacklog[i] for i in dummyVariableNamesListBacklog] + 
                  [100000000*dummyDecisionVariablesMOT[i] for i in dummyVariableNamesListMOT] + 
                  [100000000*dummyDecisionVariablesVTO[i] for i in dummyVariableNamesListVTO] + 
                  [100000000*dummyDecisionVariablesLaborshare[i] for i in dummyVariableNamesListLaborshare] +                   
                  [currentEmployeeCostWeek"""+ str(currentWeek) +""" +
                   currentEmployeeCostWeek"""+ str(currentWeek+1) +""" +
                   currentEmployeeCostWeek"""+ str(currentWeek+2) +""" +
                   currentEmployeeCostWeek"""+ str(currentWeek+3) +""" +
                   currentEmployeeCostWeek"""+ str(currentWeek+4) +""" +
                   currentEmployeeCostWeek"""+ str(currentWeek+5) +""" +
                   currentEmployeeCostWeek"""+ str(currentWeek+6) +""" +
                   currentEmployeeCostWeek"""+ str(currentWeek+7) +""" +
                   currentEmployeeCostWeek"""+ str(currentWeek+8) +""" +
                   currentEmployeeCostWeek"""+ str(currentWeek+9) +""" +
                   currentEmployeeCostWeek"""+ str(currentWeek+10) +""" +
                   currentEmployeeCostWeek"""+ str(currentWeek+11) +""" +
                   currentEmployeeCostWeek"""+ str(currentWeek+12) +""" ]) """)
    ###################################################
    ##     Defining the constraints                 ###
    ###################################################
    # week 1  
    # units capacity 
    print('generating current week constraints ....')
#     print('capacity week {}'.format(currentWeek))
    exec("""prob += lpSum(([futureEmployeeCapacityWeek"""+ str(currentWeek) +"""[i]*futureEmployeeDecisionVariables[i] for i in futureEmployeeClassNamesList] +
                   [globals()[i].trainingCapacityCoef(targetWeek = """+ str(currentWeek) +""")*(-1)*futureEmployeeDecisionVariables[i] for i in futureEmployeeClassNamesList] +
                   [globals()[i].hourIndicator(week = """+ str(currentWeek) +""")*currentEmployeeRateWeek"""+ str(currentWeek) +"""*(-1)*VTODecisionVariables[i] for i in VTOClassNamesList] +
                   [globals()[i].hourIndicator(week = """+ str(currentWeek) +""")*currentEmployeeRateWeek"""+ str(currentWeek) +"""*MOTDecisionVariables[i] for i in MOTClassNamesList] +
                   [globals()[i].hourIndicator(week = """+ str(currentWeek) +""")*(-1)*laborShareDecisionVariables[i] for i in laborShareClassNamesList] 
                   ))*(1 + 1*0.7/7)  + currentEmployeeCapacityWeek"""+ str(currentWeek) +"""*(1 + 1*0.7/7) + (
                   lpSum([dummyDecisionVariablesCapacity[v] for v in dummyDecisionVariablesCapacity if int(v.split('Week')[-1])==currentWeek])) >= (
    extract(demandDf,week="""+ str(currentWeek) +""",field ='demandActual') + extract(backlogDf,week="""+ str(currentWeek-1) +""",field ='backlogValue')), "Capacity Requirement week"""+ str(currentWeek) +""""

    # units capacity 
# print('process week {}'.format(currentWeek))
prob += lpSum(([futureEmployeeCapacityWeek"""+ str(currentWeek) +"""[i]*futureEmployeeDecisionVariables[i] for i in futureEmployeeClassNamesList] +
                   [globals()[i].trainingCapacityCoef(targetWeek = """+ str(currentWeek) +""")*(-1)*futureEmployeeDecisionVariables[i] for i in futureEmployeeClassNamesList] +                   
                   [globals()[i].hourIndicator(week = """+ str(currentWeek) +""")*currentEmployeeRateWeek"""+ str(currentWeek) +"""*(-1)*VTODecisionVariables[i] for i in VTOClassNamesList] +
                   [globals()[i].hourIndicator(week = """+ str(currentWeek) +""")*currentEmployeeRateWeek"""+ str(currentWeek) +"""*MOTDecisionVariables[i] for i in MOTClassNamesList] +
                   [globals()[i].hourIndicator(week = """+ str(currentWeek) +""")*(-1)*laborShareDecisionVariables[i] for i in laborShareClassNamesList] 
                   )) + currentEmployeeCapacityWeek"""+ str(currentWeek) +""" <= (
    extract(demandDf,week="""+ str(currentWeek) +""",field ='demandActual') + extract(backlogDf,week="""+ str(currentWeek-1) +""",field ='backlogValue'))+ (
    lpSum([dummyDecisionVariablesProcess[v] for v in dummyDecisionVariablesProcess if int(v.split('Week')[-1])==currentWeek])) ,  "process less than demand week"""+ str(currentWeek) +""""
    # units capacity 
# print('backlog week {}'.format(currentWeek))
prob += endWeekBacklogDecisionVariables['backlogWeek"""+ str(currentWeek) +"""'] == (
            extract(demandDf,week="""+ str(currentWeek) +""",field ='demandActual') + extract(backlogDf,week="""+ str(currentWeek-1) +""",field ='backlogValue') + 
    -1*(lpSum(([futureEmployeeCapacityWeek"""+ str(currentWeek) +"""[i]*futureEmployeeDecisionVariables[i] for i in futureEmployeeClassNamesList] +
           [globals()[i].trainingCapacityCoef(targetWeek = """+ str(currentWeek) +""")*(-1)*futureEmployeeDecisionVariables[i] for i in futureEmployeeClassNamesList] +              
           [globals()[i].hourIndicator(week = """+ str(currentWeek) +""")*currentEmployeeRateWeek"""+ str(currentWeek) +"""*(-1)*VTODecisionVariables[i] for i in  VTOClassNamesList] +
           [globals()[i].hourIndicator(week = """+ str(currentWeek) +""")*currentEmployeeRateWeek"""+ str(currentWeek) +"""*MOTDecisionVariables[i] for i in MOTClassNamesList] +
           [globals()[i].hourIndicator(week = """+ str(currentWeek) +""")*(-1)*laborShareDecisionVariables[i] for i in laborShareClassNamesList] 
           ))+ currentEmployeeCapacityWeek"""+ str(currentWeek) +""")) + (
           lpSum([dummyDecisionVariablesBacklog[v] for v in dummyDecisionVariablesBacklog if int(v.split('Week')[-1])==currentWeek])),  "Backlog flow balance week"""+ str(currentWeek) +""""
    # units hour
# print('MOT week {}'.format(currentWeek))
prob += [globals()[i].hourIndicator(week = """+ str(currentWeek) +""")*MOTDecisionVariables[i] for i in MOTClassNamesList] <= (
            lpSum(([futureEmployeeVeteranHourWeek"""+ str(currentWeek) +"""[i]*futureEmployeeDecisionVariables[i] for i in futureEmployeeClassNamesList] +
                   [globals()[i].trainingVetHourCoef(targetWeek = """+ str(currentWeek) +""")*(-1)*futureEmployeeDecisionVariables[i] for i in futureEmployeeClassNamesList] 
                   )) + currentEmployeeVeteranHourWeek"""+ str(currentWeek) +""")*0.1 + (
                   lpSum([dummyDecisionVariablesMOT[v] for v in dummyDecisionVariablesMOT if int(v.split('Week')[-1])==currentWeek])), "week """+ str(currentWeek) +""" MOT constraint  """+ str(currentWeek) +""" "
    # units hour 
# print('MOT week {}'.format(currentWeek))
prob += [globals()[i].hourIndicator(week = """+ str(currentWeek) +""")*VTODecisionVariables[i] for i in VTOClassNamesList] <= ( 
            lpSum(([futureEmployeeVeteranHourWeek"""+ str(currentWeek) +"""[i]*futureEmployeeDecisionVariables[i] for i in futureEmployeeClassNamesList] +
                   [globals()[i].trainingVetHourCoef(targetWeek = """+ str(currentWeek) +""")*(-1)*futureEmployeeDecisionVariables[i] for i in futureEmployeeClassNamesList]         
                   )) + currentEmployeeVeteranHourWeek"""+ str(currentWeek) +""" )*0.1 + ( 
                   lpSum([dummyDecisionVariablesVTO[v] for v in dummyDecisionVariablesVTO if int(v.split('Week')[-1])==currentWeek])), "week """+ str(currentWeek) +""" VTO constraint week """+ str(currentWeek) +""" "
# print('laborshare week {}'.format(currentWeek))
prob += lpSum([globals()[i].hourIndicator(week = """+ str(currentWeek) +""")*laborShareDecisionVariables[i] for i in laborShareClassNamesList]) <= ( 
            lpSum(([futureEmployeeCapacityWeek"""+ str(currentWeek) +"""[i]*futureEmployeeDecisionVariables[i] for i in futureEmployeeClassNamesList] +
                   [globals()[i].trainingCapacityCoef(targetWeek = """+ str(currentWeek) +""")*(-1)*futureEmployeeDecisionVariables[i] for i in futureEmployeeClassNamesList] +
                   [globals()[i].hourIndicator(week = """+ str(currentWeek) +""")*currentEmployeeRateWeek"""+ str(currentWeek) +"""*(-1)*VTODecisionVariables[i] for i in  VTOClassNamesList] +
                   [globals()[i].hourIndicator(week = """+ str(currentWeek) +""")*currentEmployeeRateWeek"""+ str(currentWeek) +"""*MOTDecisionVariables[i] for i in MOTClassNamesList] 
                   ))*(1) + currentEmployeeCapacityWeek"""+ str(currentWeek) +""" ) + (
                   lpSum([dummyDecisionVariablesLaborshare[v] for v in dummyDecisionVariablesLaborshare if int(v.split('Week')[-1])==currentWeek])) , "week """+ str(currentWeek) +""" laborShare constraint """+ str(currentWeek) +""""  """)


    # week 2 and + constraints 
    A = ["""
# print('capacity week {}'.format("""+ str(week) +"""))
prob += lpSum(([futureEmployeeCapacityWeek"""+ str(week) +"""[i]*futureEmployeeDecisionVariables[i] for i in futureEmployeeClassNamesList] +
                   [globals()[i].trainingCapacityCoef(targetWeek = """+ str(week) +""")*(-1)*futureEmployeeDecisionVariables[i] for i in futureEmployeeClassNamesList] +
                   [globals()[i].hourIndicator(week = """+ str(week) +""")*currentEmployeeRateWeek"""+ str(week) +"""*(-1)*VTODecisionVariables[i] for i in  VTOClassNamesList] +
                   [globals()[i].hourIndicator(week = """+ str(week) +""")*currentEmployeeRateWeek"""+ str(week) +"""*MOTDecisionVariables[i] for i in MOTClassNamesList] +
                   [globals()[i].hourIndicator(week = """+ str(week) +""")*(-1)*laborShareDecisionVariables[i] for i in laborShareClassNamesList] 
                   ))*(1+0.7/7) + currentEmployeeCapacityWeek"""+ str(week) +"""*(1+0.7/7) + (
                   lpSum([dummyDecisionVariablesCapacity[v] for v in dummyDecisionVariablesCapacity if int(v.split('Week')[-1])==week])) >= (
    extract(demandDf,week="""+ str(week) +""",field ='demandForecast') +  endWeekBacklogDecisionVariables['backlogWeek"""+ str(week-1) +"""'] ), "Capacity Requirement week """+ str(week) +""" "

# print('process week {}'.format("""+ str(week) +"""))
prob += lpSum(([futureEmployeeCapacityWeek""" +  str(week)  +"""[i]*futureEmployeeDecisionVariables[i] for i in futureEmployeeClassNamesList] +
                   [globals()[i].trainingCapacityCoef(targetWeek = """+ str(week) +""")*(-1)*futureEmployeeDecisionVariables[i] for i in futureEmployeeClassNamesList] +
                   [globals()[i].hourIndicator(week = """+ str(week) +""")*currentEmployeeRateWeek"""+ str(week) +"""*(-1)*VTODecisionVariables[i] for i in  VTOClassNamesList] +
                   [globals()[i].hourIndicator(week = """+ str(week) +""")*currentEmployeeRateWeek"""+ str(week) +"""*MOTDecisionVariables[i] for i in MOTClassNamesList] +
                   [globals()[i].hourIndicator(week = """+ str(week) +""")*(-1)*laborShareDecisionVariables[i] for i in laborShareClassNamesList] 
                   )) + currentEmployeeCapacityWeek"""+ str(week) +""" <= (
    extract(demandDf,week="""+ str(week) +""",field ='demandForecast') +  endWeekBacklogDecisionVariables['backlogWeek"""+ str(week-1) +"""'] ) + (
    lpSum(lpSum([dummyDecisionVariablesCapacity[v] for v in dummyDecisionVariablesCapacity if int(v.split('Week')[-1])==week]))) ,  "process less than demand week"""+ str(week) +""" "

# print('backlog week {}'.format("""+ str(week) +"""))
prob += endWeekBacklogDecisionVariables['backlogWeek"""+ str(week) +"""'] == (
            extract(demandDf,week="""+ str(week) +""",field ='demandForecast') +  endWeekBacklogDecisionVariables['backlogWeek"""+ str(week-1) +"""'] + 
    -1*(lpSum(([futureEmployeeCapacityWeek"""+ str(week) +"""[i]*futureEmployeeDecisionVariables[i] for i in futureEmployeeClassNamesList] +
           [globals()[i].trainingCapacityCoef(targetWeek = """+ str(week) +""")*(-1)*futureEmployeeDecisionVariables[i] for i in futureEmployeeClassNamesList] +    
           [globals()[i].hourIndicator(week = """+ str(week) +""")*currentEmployeeRateWeek"""+ str(week) +"""*(-1)*VTODecisionVariables[i] for i in  VTOClassNamesList] +
           [globals()[i].hourIndicator(week = """+ str(week) +""")*currentEmployeeRateWeek"""+ str(week) +"""*MOTDecisionVariables[i] for i in MOTClassNamesList] +
           [globals()[i].hourIndicator(week = """+ str(week) +""")*(-1)*laborShareDecisionVariables[i] for i in laborShareClassNamesList] 
           ))+ currentEmployeeCapacityWeek"""+ str(week) +""") + (
lpSum([dummyDecisionVariablesBacklog[v] for v in dummyDecisionVariablesBacklog if int(v.split('Week')[-1])==currentWeek]))),  "Backlog flow balance week"""+ str(week) +""" "

# print('MOT week {}'.format("""+ str(week) +"""))
prob += [globals()[i].hourIndicator(week = """+ str(week) +""")*MOTDecisionVariables[i] for i in MOTClassNamesList] <= (
            lpSum(([futureEmployeeVeteranHourWeek"""+ str(week) +"""[i]*futureEmployeeDecisionVariables[i] for i in futureEmployeeClassNamesList] +
                   [globals()[i].trainingVetHourCoef(targetWeek = """+ str(week) +""")*(-1)*futureEmployeeDecisionVariables[i] for i in futureEmployeeClassNamesList] 
                   )) +currentEmployeeVeteranHourWeek"""+ str(week) +""" )*0.1 + (
                   lpSum([dummyDecisionVariablesMOT[v] for v in dummyDecisionVariablesMOT if int(v.split('Week')[-1])==week])), "week """+ str(week) +""" MOT constraint"

# print('VTO week {}'.format("""+ str(week) +"""))
prob += [globals()[i].hourIndicator(week = """+ str(week) +""")*VTODecisionVariables[i] for i in VTOClassNamesList] <= ( 
            lpSum(([futureEmployeeVeteranHourWeek"""+ str(week) +"""[i]*futureEmployeeDecisionVariables[i] for i in futureEmployeeClassNamesList] +
                   [globals()[i].trainingVetHourCoef(targetWeek = """+ str(week) +""")*(-1)*futureEmployeeDecisionVariables[i] for i in futureEmployeeClassNamesList] 
                  )) +  currentEmployeeVeteranHourWeek"""+ str(week) +""" )*0.1 + (
                  lpSum([dummyDecisionVariablesVTO[v] for v in dummyDecisionVariablesVTO if int(v.split('Week')[-1])==week])), "week """+ str(week) +""" VTO constraint"

# print('laborshare week {}'.format("""+ str(week) +"""))
prob += lpSum([globals()[i].hourIndicator(week = """+ str(week) +""")*laborShareDecisionVariables[i] for i in laborShareClassNamesList]) <= ( 
            lpSum(([futureEmployeeCapacityWeek"""+ str(week) +"""[i]*futureEmployeeDecisionVariables[i] for i in futureEmployeeClassNamesList] +
                   [globals()[i].trainingCapacityCoef(targetWeek = """+ str(week) +""")*(-1)*futureEmployeeDecisionVariables[i] for i in futureEmployeeClassNamesList] +
                   [globals()[i].hourIndicator(week = """+ str(week) +""")*currentEmployeeRateWeek"""+ str(week) +"""*(-1)*VTODecisionVariables[i] for i in  VTOClassNamesList] +
                   [globals()[i].hourIndicator(week = """+ str(week) +""")*currentEmployeeRateWeek"""+ str(week) +"""*MOTDecisionVariables[i] for i in MOTClassNamesList] 
                   ))*(1) + currentEmployeeCapacityWeek"""+ str(week) +""" ) + (
                   lpSum([dummyDecisionVariablesLaborshare[v] for v in dummyDecisionVariablesLaborshare if int(v.split('Week')[-1])==week])), "week """+ str(week) +""" laborShare constraint" """ for week in range(currentWeek+1,currentWeek+13)]

    for i in A:
        exec(i)

    prob.solve()
#     costs[run] = value(prob.objective)
#     costs[str(simulation)+ 'run '+str(run)] = value(prob.objective)
    print("Status:", LpStatus[prob.status])
#     costs2.append((str(counter)+ ''+ str(simulation)+ 'run '+str(run), value(prob.objective)))

#     print("Total Cost  = ", value(prob.objective))
#     print(str(simulation)+' run '+str(run), "current employee capacity =", int(currentEmployeeCapacityWeek1))
    #         if simulation == 'rate':
    if currentWeek > 3:
        print('current week cost {}'.format(labor_cost(currentWeek)))
        totalCost= totalCost+labor_cost(currentWeek)
        print('total cost {}'.format(totalCost))
#     print(demandDf[demandDf.week == currentWeek])
#     print(attritionForecastsDf[attritionForecastsDf.week == currentWeek])
#     print(attendanceForecastsDf[attendanceForecastsDf.week == currentWeek]) 
#     print([(i,i.varValue) for i in prob.variables()])
    headcountDf = headcountDfUpdater(currentWeek,headcountDf)
    backlogDf = BacklogUpdater(currentWeek,backlogDf)
    
#     plotter(futureEmployeeClassNamesList)
#     plotter(MOTClassNamesList)
#     plotter(endWeekBacklogVariableNamesList)
#     plotter(VTOClassNamesList)
#     plotter(laborShareClassNamesList)
#     plotter(dummyVariableNamesListCapacity)
#     plotter(dummyVariableNamesListProcess)
#     plotter(dummyVariableNamesListBacklog)
#     plotter(dummyVariableNamesListMOT)
#     plotter(dummyVariableNamesListVTO)
#     plotter(dummyVariableNamesListLaborshare)
    
    
    

#     plotter(backlogDf)
#     print([(i,i.varValue) for i in prob.variables()])

#     print('backlog :',backlogDf)
# total cost 2101387421.53
#     solution.append([(i,i.varValue) for i in prob.variables()])

atten 0.0 attr -0.0 dem 1.0 rat 1.0
run  0  for  attrition
attrition error : 1.36130928771
attendance error : 3.34123114818
running optimization for week 1 .... 


/home/pemfir/anaconda3/envs/CarND-LeNet-Lab/lib/python3.5/site-packages/ipykernel/__main__.py:125: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/home/pemfir/anaconda3/envs/CarND-LeNet-Lab/lib/python3.5/site-packages/ipykernel/__main__.py:157: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


generating objective function ....
generating current week constraints ....
Status: Optimal


In [58]:
prob.writeLP

<bound method LpProblem.writeLP of Labor Planning Basic:
MINIMIZE
26.25*MOT_MOTeight + 26.25*MOT_MOTeleven + 26.25*MOT_MOTfive + 26.25*MOT_MOTfour + 26.25*MOT_MOTnine + 26.25*MOT_MOTone + 26.25*MOT_MOTseven + 26.25*MOT_MOTsix + 26.25*MOT_MOTten + 26.25*MOT_MOTthirteen + 26.25*MOT_MOTthree + 26.25*MOT_MOTtwelve + 26.25*MOT_MOTtwo + -15.0*VTO_VTOeight + -15.0*VTO_VTOeleven + -15.0*VTO_VTOfive + -15.0*VTO_VTOfour + -15.0*VTO_VTOnine + -15.0*VTO_VTOone + -15.0*VTO_VTOseven + -15.0*VTO_VTOsix + -15.0*VTO_VTOten + -15.0*VTO_VTOthirteen + -15.0*VTO_VTOthree + -15.0*VTO_VTOtwelve + -15.0*VTO_VTOtwo + 1e-05*backlog_backlogWeek1 + 1e-05*backlog_backlogWeek10 + 1e-05*backlog_backlogWeek11 + 1e-05*backlog_backlogWeek12 + 1e-05*backlog_backlogWeek13 + 1e-05*backlog_backlogWeek2 + 1e-05*backlog_backlogWeek3 + 1e-05*backlog_backlogWeek4 + 1e-05*backlog_backlogWeek5 + 1e-05*backlog_backlogWeek6 + 1e-05*backlog_backlogWeek7 + 1e-05*backlog_backlogWeek8 + 1e-05*backlog_backlogWeek9 + 100000000*dummyBack

In [56]:
employeeAMZeight.effectiveSurvival(20)

0.8069555730306066

In [43]:
attendanceFunc(3,8,5,'AMZ') # prediction depends on currentWeek


24    0.801547
Name: predictedRate, dtype: float64

In [37]:
attendanceDataFrame(5,attendanceForecastsDf,0)

,week,laborType,hoursWorked,hoursScheduled,actualRate,predictedRate,error
0,-4,AMZ,79.5031358912383,100,0.795031,0.795031,NaN
1,-4,TEMP,70.15968462601725,100,0.701597,0.701597,NaN
2,-3,AMZ,78.76458670400596,100,0.787646,0.787646,-0.007385
3,-3,TEMP,69.20007083860696,100,0.692001,0.692001,-0.009596
4,-2,AMZ,78.63423200710133,100,0.786342,0.786342,-0.004996
5,-2,TEMP,67.94429888263619,100,0.679443,0.679443,-0.017356
6,-1,AMZ,80.87251520868183,100,0.808725,0.808725,0.019052
7,-1,TEMP,68.83056357747843,100,0.688306,0.688306,-0.002708
8,0,AMZ,76.59397445177352,100,0.765940,0.765940,-0.028496
9,0,TEMP,71.63318785309568,100,0.716332,0.716332,0.025995


In [11]:
currentWeek = 5
np.prod(1- attritionForecastsDf[(attritionForecastsDf.week <= currentWeek) & (attritionForecastsDf.week >= 0)].predictedRate)

0.5572517896818413

In [ ]:
currentWeek = 1

attritionUpdatedDf = pysqldf("""
    SELECT 
     a.week
    ,a.laborType
    ,a.attritionCount
    ,a.headcount
    ,a.actualRate
    ,CASE 
        WHEN a.week <= """ + str(currentWeek) + """ THEN a.actualRate
        ELSE b.predictedRate END as predictedRate
    ,CASE 
        WHEN a.week <= """ + str(currentWeek) + """ THEN a.actualRate - a.predictedRate
        ELSE a.actualRate - b.predictedRate END as error
    FROM attritionForecastsDf a
    LEFT JOIN (SELECT * FROM attritionForecastsDf b WHERE b.week = """ + str(currentWeek) + """) b
    ON a.laborType = b.laborType 
    order by a.week ; """) 

In [ ]:
pysqldf("""
    SELECT 
     * from attritionUpdatedDf """)

In [ ]:
type(attritionForecastsDf)

In [ ]:
type(attritionUpdatedDf)

In [4]:
attritionForecastsDf

,week,laborType,attritionCount,headcount,actualRate,predictedRate,error
0,-4,AMZ,2.0257636662326846,100.0,0.020258,NaN,NaN
1,-4,TEMP,9.60215977180001,100.0,0.096022,NaN,NaN
2,-3,AMZ,2.9296551228958974,100.0,0.029297,0.000258,0.009039
3,-3,TEMP,10.337437653613973,100.0,0.103374,0.076022,0.007353
4,-2,AMZ,3.3079688552160342,100.0,0.033080,0.004777,0.008303
5,-2,TEMP,11.047578572892721,100.0,0.110476,0.079698,0.010778
6,-1,AMZ,2.79150123689412,100.0,0.027915,0.007545,0.000370
7,-1,TEMP,11.045938255627664,100.0,0.110459,0.083291,0.007169
8,0,AMZ,4.033800732555499,100.0,0.040338,0.007637,0.012701
9,0,TEMP,10.86371729168484,100.0,0.108637,0.085083,0.003554


In [6]:
survivalDataFrame(4,attritionForecastsDf,1)

,week,laborType,attritionCount,headcount,actualRate,predictedRate,error
0,-4,AMZ,2.0257636662326846,100.0,0.020258,0.020258,NaN
1,-4,TEMP,9.60215977180001,100.0,0.096022,0.096022,NaN
2,-3,AMZ,2.9296551228958974,100.0,0.029297,0.029297,0.029039
3,-3,TEMP,10.337437653613973,100.0,0.103374,0.103374,0.027353
4,-2,AMZ,3.3079688552160342,100.0,0.033080,0.033080,0.028303
5,-2,TEMP,11.047578572892721,100.0,0.110476,0.110476,0.030778
6,-1,AMZ,2.79150123689412,100.0,0.027915,0.027915,0.020370
7,-1,TEMP,11.045938255627664,100.0,0.110459,0.110459,0.027169
8,0,AMZ,4.033800732555499,100.0,0.040338,0.040338,0.032701
9,0,TEMP,10.86371729168484,100.0,0.108637,0.108637,0.023554


In [ ]:
att

In [ ]:
!pip install pandasql


In [ ]:
prob.writeLP

In [ ]:
backlogDf

In [ ]:
totalCost

In [ ]:
labor_cost(currentWeek)

In [ ]:
totalCost= totalCost+labor_cost(currentWeek)

In [ ]:
print('total cost',totalCost+=labor_cost(currentWeek))

In [ ]:
A = 0
A += 1
A

In [ ]:
totalCost+=labor_cost(currentWeek)

In [ ]:
totalCost